In [1]:
# List combine PDFs

In [2]:
## Instalar
### pip install openpyxl PyPDF2

In [3]:

import os
import threading
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from openpyxl import Workbook, load_workbook
from PyPDF2 import PdfMerger

# Función para listar archivos PDF en una carpeta
def listar_archivos(ruta):
    archivos = []
    for root, dirs, files in os.walk(ruta):
        for file in files:
            if file.endswith(".pdf"):  # Solo agregamos los archivos PDF
                archivos.append(file)  # Solo el nombre del archivo, sin la ruta
    return archivos

# Función para generar un archivo Excel con los PDFs encontrados
def generar_excel_listado(archivos_pdf, nombre_archivo):
    wb = Workbook()
    ws = wb.active
    ws.title = "Listado de PDFs"

    # Agregar encabezados
    ws.append(['PDF-BASE', 'PDF-1', 'PDF-2', 'RESULTADO'])

    # Llenar la columna PDF-BASE con los archivos encontrados
    for pdf in archivos_pdf:
        ws.append([pdf, '', '', ''])  # Dejar PDF-1, PDF-2 y RESULTADO vacíos

    # Guardar el archivo Excel
    wb.save(nombre_archivo)
    messagebox.showinfo("Éxito", f"Archivo Excel generado: {nombre_archivo}")

# Función para seleccionar carpeta de PDFs y generar un listado en Excel
def seleccionar_carpeta_pdfs():
    ruta = filedialog.askdirectory(title="Seleccionar la carpeta que contiene los archivos PDF")
    if ruta:
        archivos_pdf = listar_archivos(ruta)
        if not archivos_pdf:
            messagebox.showwarning("Advertencia", "No se encontraron archivos PDF en la carpeta.")
            return
        
        # Guardar el listado en un archivo Excel
        nombre_excel = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx")], title="Guardar el listado de PDFs")
        if nombre_excel:
            generar_excel_listado(archivos_pdf, nombre_excel)
        return ruta
    return None

# Función para unir PDFs según lo indicado en el archivo Excel
def unir_pdfs(excel_file, ruta_salida, ruta_base, progress_bar):
    try:
        wb = load_workbook(excel_file)
        ws = wb.active
        
        total_rows = ws.max_row - 1  # Ignorar encabezado
        current_row = 0

        for row in ws.iter_rows(min_row=2, values_only=True):  # Ignorar la fila de encabezado
            pdf_base = row[0]
            pdf1 = row[1]
            pdf2 = row[2]
            resultado = row[3]

            # Si el campo resultado está vacío, generar un nombre automático
            if not resultado:
                resultado = f"{pdf1.replace('.pdf', '')}_{pdf2.replace('.pdf', '')}_combinado.pdf"

            # Validar que los archivos existan
            if not pdf1 or not pdf2:
                messagebox.showwarning("Advertencia", f"Faltan archivos en la fila con base {pdf_base}. Saltando esta fila.")
                continue

            pdf1_path = os.path.join(ruta_base, pdf1)
            pdf2_path = os.path.join(ruta_base, pdf2)
            resultado_path = os.path.join(ruta_salida, resultado)

            if not os.path.exists(pdf1_path) or not os.path.exists(pdf2_path):
                messagebox.showerror("Error", f"Los archivos {pdf1} o {pdf2} no existen en la carpeta base.")
                continue

            # Combinar los PDFs
            merger = PdfMerger()
            with open(pdf1_path, 'rb') as f1, open(pdf2_path, 'rb') as f2:
                merger.append(f1)
                merger.append(f2)

            # Guardar el PDF combinado
            with open(resultado_path, 'wb') as f_result:
                merger.write(f_result)
            
            merger.close()
            current_row += 1

            # Actualizar la barra de progreso
            progress_bar['value'] = (current_row / total_rows) * 100
            root.update_idletasks()
        
        messagebox.showinfo("Éxito", "Archivos PDF combinados correctamente.")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Función para seleccionar archivo Excel
def seleccionar_excel():
    ruta_excel = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")], title="Seleccionar el archivo Excel")
    if ruta_excel:
        return ruta_excel
    return None

# Función para seleccionar carpeta de salida
def seleccionar_carpeta_salida():
    ruta_salida = filedialog.askdirectory(title="Seleccionar la carpeta de salida")
    if ruta_salida:
        return ruta_salida
    return None

# Función principal para iniciar el proceso en un hilo separado
def iniciar_proceso():
    ruta_excel = seleccionar_excel()
    if not ruta_excel:
        return

    ruta_salida = seleccionar_carpeta_salida()
    if not ruta_salida:
        return

    ruta_base = filedialog.askdirectory(title="Seleccionar la carpeta base donde están los archivos PDF")
    if not ruta_base:
        return

    # Mostrar barra de progreso
    progress_bar = ttk.Progressbar(root, orient="horizontal", length=300, mode="determinate")
    progress_bar.pack(pady=20)

    # Ejecutar el proceso de combinación en un hilo separado
    threading.Thread(target=unir_pdfs, args=(ruta_excel, ruta_salida, ruta_base, progress_bar)).start()

# Función para salir de la aplicación
def salir():
    root.quit()
    root.destroy()

# Crear la interfaz gráfica
root = tk.Tk()
root.title("List combine PDFs")
root.geometry("400x400")

# Crear botones
btn_listar_archivos = tk.Button(root, text="Paso 1: Seleccionar carpeta de PDFs", command=seleccionar_carpeta_pdfs)
btn_comenzar_union = tk.Button(root, text="Paso 2: Seleccionar Excel y Combinar PDFs", command=iniciar_proceso)
btn_salir = tk.Button(root, text="Salir", command=salir, fg="white", bg="red")  # Botón de salida

# Barra de progreso (inicialmente oculta)
progress_bar = None

# Ubicar botones en la ventana
btn_listar_archivos.pack(pady=20)
btn_comenzar_union.pack(pady=20)
btn_salir.pack(pady=20)  # Añadido botón de salida

# Copyright en la parte inferior
label_copyright = tk.Label(root, text="Copyright @Jorge A Melo - 2022", font=("Arial", 10))
label_copyright.pack(side="bottom", pady=10)

# Iniciar la aplicación
root.mainloop()

In [ ]:
## FIN